In [47]:
# Aula prática de áudio digital
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import resample
from IPython.display import Audio, display
import ipywidgets as widgets
import os


In [48]:

def process_audio_advanced(filename, target_rate=40000, bits=16, initial_time=0, duration=5):
    # Ler arquivo original:
    # - original_rate -> taxa de amostragem em Hz
    # - audio_data -> um array NumPy contendo as amostras do áudio.
    # - audio_data.shape (1 ou 2): número de canais: 1D para mono e 2D para estéreo
    original_rate, audio_data = wavfile.read(filename)

    # Se estéreo, pegar apenas um canal
    if len(audio_data.shape) > 1:
        audio_data = audio_data[:,0]
        # Seleciona todas as linhas (:) e apenas a primeira coluna (0), ou seja, o canal esquerdo.

    # Altera a taxa de amostragem do áudio (resampling),
    # len(audio_data) → número total de amostras no áudio original.
    # original_rate → taxa de amostragem do áudio original (em Hz).
    # target_rate → nova taxa de amostragem desejada (em Hz).
    # Ajusta o número de amostras do áudio para a nova taxa de amostragem.
    num_samples = int(len(audio_data) * target_rate / original_rate)

    # gera o novo áudio com a taxa de amostragem selecionada
    audio_resampled = resample(audio_data, num_samples)

    #determina o maior valor de amostra
    max_val = np.max(np.abs(audio_resampled))

    # Normaliza o áudio:
    # - Divide cada amostra pelo valor absoluto máximo (max_val) do áudio resampleado.
    # - Resultado: valores entre -1 e 1, independentemente da amplitude original.
    audio_normalized = audio_resampled / max_val

    max_val = np.max(np.abs(audio_resampled))
    if bits == 8:
        # Converte para byte entre [0,255], onde 127 é zero
        audio_processed = np.uint8(audio_normalized*127 + 128)
    elif bits == 16:
        # Converte para int16 entre [-32767, +32767]
        audio_processed = np.int16(audio_normalized * 32767)
    elif bits == 4:
        # audio_processed: valores inteiros de -7 até +7 (15 níveis)
        audio_processed = np.round(audio_normalized * 7)
        # Reescala para [0, 14]
        audio_processed = audio_processed + 7
        # Mapeia para [0, 255]
        audio_processed = np.uint8(audio_processed * (255 // 14))
    else:
        raise ValueError("Bits permitidos: 4, 8 ou 16")

    # Salvar arquivo
    base, ext = os.path.splitext(filename)
    output_filename = f"{base}_{target_rate}Hz_{bits}bit.wav"
    wavfile.write(output_filename, target_rate, audio_processed)
    print(f"Arquivo salvo como: {output_filename}")

    if bits == 8 or bits == 4:
        # Antes de calcular a FFT, converta para float centrado em zero
        audio_processed = audio_processed.astype(np.float32)        # converte para float
        audio_processed = audio_processed - 128                     # remove offset DC
    fft_vals = np.fft.rfft(audio_processed)
    fft_freq = np.fft.rfftfreq(len(audio_processed), 1/target_rate)
    magnitude = np.abs(fft_vals)


    # Plot waveform
    plt.figure(figsize=(12, 3))
    time_axis = np.linspace(0, len(audio_processed)/target_rate, num=len(audio_processed))
    plt.plot(time_axis, audio_processed)
    plt.title(f"Waveform - Rate: {target_rate} Hz, Bits: {bits}")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.show()

    # Plot espectro de frequência
    plt.figure(figsize=(12, 3))
    plt.plot(fft_freq, magnitude)
    plt.title(f"Frequency Spectrum")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Magnitude")
    plt.show()

    # Plot waveform
    plt.figure(figsize=(12, 3))
    time_axis = np.linspace(0, len(audio_processed)/target_rate, num=len(audio_processed))
    plt.plot(time_axis, audio_processed)
    plt.title(f"Waveform - Rate: {target_rate} Hz, Bits: {bits}")
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude")
    plt.xlim([initial_time, initial_time + duration]) # Limit x-axis to 0-1 seconds
    plt.show()

    # Reproduzir áudio
    display(Audio(audio_processed, rate=target_rate))



In [49]:
    # Arquivo de áudio
    filename = "audio.wav"

In [50]:


# Widget interativo
widgets.interact(
    process_audio_advanced,
    filename=widgets.fixed(filename),
    target_rate=widgets.IntSlider(min=4000, max=44100, step=1000, value=8000, description="Sampling Rate"),
    bits=widgets.Dropdown(options=[4, 8, 16], value=8, description="Bits per Sample")
)

interactive(children=(IntSlider(value=8000, description='Sampling Rate', max=44100, min=4000, step=1000), Drop…

<function __main__.process_audio_advanced(filename, target_rate=40000, bits=16, initial_time=0, duration=5)>

Procedimentos:

1) Entre no colab https://colab.research.google.com/ e abra um notebook com o código abaixo e faça o upload do arquivo audio.wav. Este arquivo tem uma taxa de amostragem de 44.1Khz, 16 bits/amostra e é mono.

2) Estude o código python

3) Responda as questões abaixo:

A) Vá no notebook e altere o som para manter os parâmetros próximos do original: 40kHz e 16 bits por amostra. Em seguida responda:

i) Qual é a maior frequência de som no arquivo?

ii) Indique o maior componente de frequência (em Hertz) possível quando estes parâmetros de digitalização são utilizados.

iii) Qual é o tamanho teórico do áudio (parte de dados);

iv) O tamanho do arquivo em bytes (ver propriedades do arquivo, ou Linux utilize o comando "ls -l audio.wav) e indique o motivo da diferença entre este tamanho e o calculado em ii).

v) O tamanho do arquivo em disco em bytes, observando as propriedades do arquivo, ou no linux utilize "du -s -B1 audio.wav", e indique o motivo da diferença entre este tamanho e o tamanho do arquivo em ii).

vi) Indique o o tamanho deste arquivo em disco se  o seu HD fosse formatado para um tamanho de bloco (unidade de alocação em disco) de 2048 B.

### Alterar som para 40khz e 16 bits

In [ ]:
# Widget interativo
widgets.interact(
    process_audio_advanced,
    filename=widgets.fixed(filename),
    target_rate=widgets.IntSlider(min=4000, max=44100, step=1000, value=40000, description="Sampling Rate"),
    bits=widgets.Dropdown(options=[4, 8, 16], value=16, description="Bits per Sample"),
    initial_time=widgets.FloatSlider(min=0, max=30, step=1, value=0, description="Initial Time (s)"),
    duration=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description="Duration (s)")
)

interactive(children=(IntSlider(value=40000, description='Sampling Rate', max=44100, min=4000, step=1000), Dro…

<function __main__.process_audio_advanced(filename, target_rate=40000, bits=16, initial_time=0, duration=5)>

In [53]:
# i) Examine the frequency spectrum plot from the previous step.
# Based on visual inspection of the plot, the highest frequency with significant magnitude appears to be around 18000 Hz.

# ii) Calculate the Nyquist frequency for a sampling rate of 40kHz.
sampling_rate = 40000  # Hz
nyquist_frequency = sampling_rate / 2
print(f"ii) The Nyquist frequency for a sampling rate of {sampling_rate} Hz is: {nyquist_frequency} Hz")

# iii) Calculate the theoretical size of the audio data.
# To do this, we need the duration of the original audio.
# We can get this from the original_rate and the length of audio_data before resampling.
original_rate, audio_data_original = wavfile.read(filename)
if len(audio_data_original.shape) > 1:
    audio_data_original = audio_data_original[:,0]
duration = len(audio_data_original) / original_rate

target_rate = 40000  # Hz
bits_per_sample = 16
bytes_per_sample = bits_per_sample / 8

theoretical_size_bytes = duration * target_rate * bytes_per_sample
print(f"iii) The theoretical size of the audio data is: {theoretical_size_bytes} bytes")

ii) The Nyquist frequency for a sampling rate of 40000 Hz is: 20000.0 Hz
iii) The theoretical size of the audio data is: 2400000.0 bytes


## ## alterar a taxa de amostragem para 10kHz,   e mantendo 16-bits por amostra


In [ ]:
# Widget interativo
widgets.interact(
    process_audio_advanced,
    filename=widgets.fixed(filename),
    target_rate=widgets.IntSlider(min=4000, max=44100, step=1000, value=10000, description="Sampling Rate"),
    bits=widgets.Dropdown(options=[4, 8, 16], value=16, description="Bits per Sample"),
    initial_time=widgets.FloatSlider(min=0, max=30, step=1, value=0, description="Initial Time (s)"),
    duration=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description="Duration (s)")
)

## alterar a taxa de amostragem para 4kHz   e mantendo 4-bits por amostra

In [ ]:
# Widget interativo
widgets.interact(
    process_audio_advanced,
    filename=widgets.fixed(filename),
    target_rate=widgets.IntSlider(min=4000, max=44100, step=1000, value=4000, description="Sampling Rate"),
    bits=widgets.Dropdown(options=[4, 8, 16], value=4, description="Bits per Sample"),
    initial_time=widgets.FloatSlider(min=0, max=30, step=1, value=0, description="Initial Time (s)"),
    duration=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=1, description="Duration (s)")
)